Copyright (c) Microsoft Corporation. Licensed under the MIT license.

### LAB02 - Introduction to Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to interact with Large Language Models. We'll complete the following steps:

1) Import general dependencies.
2) Using Semantic Kernel SDK to run prompts on a Text Completion model from Azure OpenAI. 
3) Instantiate a Kernel to add LLM capabilities to your application.
4) Connect the Kernel to a Completion model to run your first semantic function.
5) Run Chat Completion prompts using Semantic Kernel.

**Important:** We're a using a method to automatically read sensitive parameters from a `.env` file. Therefore, please ensure that you have a `.env` file in the same directory of this notebook with the following parameters:

``` python
AZURE_OPENAI_DEPLOYMENT_NAME="NAME OF YOUR MODEL DEPLOYMENT"
AZURE_OPENAI_ENDPOINT="https://<YOUR_ENDPOINT>.openai.azure.com/"
AZURE_OPENAI_API_KEY="KEY"
```
Optionally, you can update the values and run the cell below to create your `.env` file. Please note the deployment name as we'll use both **Text Completion** and **Chat Completion** in this LAB. Initially, we'll use a TEXT COMPLETION model.

<small>Copyright (c) Microsoft Corporation.
Licensed under the MIT license.</small>

In [1]:
%%writefile .env
AZURE_OPENAI_DEPLOYMENT_NAME=""
AZURE_OPENAI_ENDPOINT=""
AZURE_OPENAI_API_KEY=""

Overwriting .env


#### Step 1 - Import Dependencies

**Semantic Kernel** is a SDK currently available in C#, Java and Python. However, each language support different at the current stage. Please visit [supported languages](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) to check which features are supported per language.

In this lab, we'll use Python SDK which can be obtained in Pypi repository.

**Important:** if you get an error running the code below, make sure that you've completed the setup process. Optionally, you can run `pip install semantic_kernel` in your terminal or `!python -m pip install semantic_kernel` in a Python code cell.

In [2]:
#Importing the Semantic Kernel SDK that will be used in this lab. 
#The other dependencies will be imported directly in the step where they are used.
import semantic_kernel as sk

#### Step 2 - Using Semantic Kernel SDK to run prompts on a Text Completion model from Azure OpenAI

In this step, we'll connect to an **Azure OpenAI Service** and use a Text Completion model. 
The Semantic Kernel also supports **OpenAI**. Other AI Services, like **Hugging Face** will be supported shortly. You can visit this [link](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages#ai-service-endpoints) to check the most updated list.

In this example, we're using **AzureTextCompletion** module to interact with Text Completion models from Azure OpenAI, such as `gpt-35-turbo-instruct` or `text-davinci-003`. This is suitable for text generation activities. We are also using a module to control the behavior of the requests, defining parameters like Temperature, max_tokens and even how many responses we want to generate.

In [3]:
#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion
#Importing the module that contains the settings for the Azure Text Completion service
from semantic_kernel.connectors.ai import CompleteRequestSettings

# Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

# Create the Azure Text Completion service connector
azure_text_completion = AzureTextCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key)

# Create the settings to control the behavior to the requests of Azure Text Completion service
request_settings = CompleteRequestSettings(
    max_tokens=20,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=2
)

Now that we have all set, let's run a simple prompt to generate taglines using the Text Completion model. Please note that the number of results, response lenght and temperature are controlled by the **CompleteRequestSettings**.

In [4]:
prompt = "Write tagline for a telco company."

#Calling the Azure Text Completion service using Semantic Kernel SDK and AI Service Connector
results = await azure_text_completion.complete_async(prompt, request_settings)

#Printing out the results
i = 1
for result in results:
    print(f"Result {i}: {result}")
    i += 1

Result 1: 

"Connecting you to the world, one call at a time."
Result 2: 

"Connecting you to the world, one call at a time."


Now that we know how to use the SDK to run a prompt, let's start building out our Call Center Analytics Solution. We have already explored how to use prompts to summarize and extract relevant information from call center conversations. As our next step, let's run them in Semantic Kernel. We'll explore different methods.

We'll use the conversation below as an example. If you prefer, just change it.

In [5]:
#Example of a call center conversation between a customer and an agent. You can replace by your own example
input = """
Agent: Hello, welcome to Telco's customer service. My name is Juan, how can I assist you?
Client: Hello, Juan. I'm calling because I'm having issues with my mobile data plan. It's very slow and I can't browse the internet or use my apps.
Agent: I'm very sorry for the inconvenience, sir. Could you please tell me your phone number and your full name?
Client: Yes, sure. My number is 011-4567-8910 and my name is Martín Pérez.
Agent: Thank you, Mr. Pérez. I'm going to check your plan and your data usage. One moment, please.
Client: Okay, thank you.
Agent: Mr. Pérez, I've reviewed your plan and I see that you have the basic plan of 2 GB of data per month. Is that correct?
Client: Yes, that's right.
Agent: Well, I inform you that you have consumed 90% of your data limit and you only have 200 MB available until the end of the month. That's why your browsing speed has been reduced.
Client: What? How is that possible? I barely use the internet on my cell phone. I just check my email and my social networks from time to time. I don't watch videos or download large files.
Agent: I understand, Mr. Pérez. But keep in mind that some applications consume data in the background, without you realizing it. For example, automatic updates, backups, GPS, etc.
Client: Well, but they didn't explain that to me when I contracted the plan. They told me that with 2 GB I would have enough for the whole month. I feel cheated.
Agent: I apologize, Mr. Pérez. It was not our intention to deceive you. I offer you a solution: if you want, you can change your plan to a higher one, with more GB of data and higher speed. This way you can enjoy a better browsing experience.
Client: And how much would that cost me?
Agent: We have a special offer for you. For only 10 pesos more per month, you can access the premium plan of 5 GB of data and 4G speed. Are you interested?
Client: Mmm, I don't know. Isn't there another option? Can't you give me more speed without charging me more?
Agent: I'm sorry, Mr. Pérez. That's the only option we have available. If you don't change your plan, you'll have to wait until next month to recover your normal speed. Or you can buy an additional data package, but it would be more expensive than changing plans.
Client: Well, let me think about it. Can I call later to confirm?
Agent: Of course, Mr. Pérez. You can call whenever you want. The number is the same one you dialed now. Is there anything else I can help you with?
Client: No, that's all. Thank you for your attention.
Agent: Thank you, Mr. Pérez. Have a good day. Goodbye.
"""

Let's run the same prompt for "Problem Summarization". Note that we're using python strings to append the Conversation to our prompt. As you run this cell, we should see a summary similar to what we saw in Azure AI Studio. 

In [6]:
#Initially, let's use a prompt similar to what we have tested in the AI Studio.
prompt = f"""
Summarize the problem described below. 
Be brief, yet informative.
Try to capture in the summary:
- The problem description;
- The product customer is complaining (if available);
- How Agente, handled the call;
- The outcome of the call. 

[INPUT]
{input}
[END INPUT]
"""

# Create the settings to control the behavior to the requests of Azure Text Completion service
request_settings = CompleteRequestSettings(
    max_tokens=200,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=1
)

#Calling the Azure Text Completion service using Semantic Kernel SDK and AI Service Connector
result = await azure_text_completion.complete_async(prompt, request_settings)

print(f"Result: {result}")

Result: 
Customer has slow internet and data issues, requests help from Agente. Agente verifies customer's plan and data usage, explains that they have reached their limit and suggests upgrading for a better experience. Customer feels misled and asks for other options, but is offered a special upgrade deal or the option to wait until next month or purchase an additional data package. Customer wants time to consider and can call back later. Outcome is unresolved, customer thanked Agente for assistance.


#### Step 3 - Instantiate a Kernel to add LLM capabilities to your application
The core of Semantic Kernel is it's Kernel. It's responsible to transform an User Input (or ASK) into a response, optionally orchestrating Plugins in this process. It's also what makes Semantic Kernel shines as it can transform existing applications in AI applications.

In this step, we'll instantiate a simple Kernel with no additonal options. Optionally, we could define Logging, Memory and the set of Plugins we want to use. 

In [7]:
#Instantiating the kernel
kernel = sk.Kernel()


#Example of how to instantiate the kernel with a different log level
#logger = sk.NullLogger()
#kernel_with_logging = sk.Kernel(log=logger)

#### Step 4 - Connect the Kernel to a Completion model to run your first semantic function.
Now that we have our kernel, we can start adding capabilities to it. We already saw how to use the SDK and connectors to the AI services to run prompts. Now, let's see how can we use the Kernel to transform a prompt into a semantic function able to run Text Completion activities using a LLM.

The first step is to add to our kernel the Text Completion model that we already have defined before. We can add multiple models and give then names (or nicknames) to later decide which ones we want to use and define programmatically which one is the default.

You can also add Chat Completion models, like GPT-4 and GPT-3.5-Turbo, as per the commented code.

In [8]:
# Add the text completion service to the kernel
kernel.add_text_completion_service("Text Completion", 
                                   AzureTextCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key))

#We can add several models of the same type to the kernel
#kernel.add_text_completion_service("Other Text Completion",
#                                   AzureTextCompletion("text-curie-001", endpoint, api_key))

#You can also define a default Text Completion / Chat Completion service to be used by the kernel 
#kernel.set_default_text_completion_service("Text Completion")

With the kernel connected to a large language model from Azure OpenAI Service, we can start using it to run Text Completion operations. The best way to make our prompts reusable is to create a **Semantic Function**. You can create and add one to the Kernel based on a **Prompt**. This is simple a textual description, or instruction, of what we want the function to do. Optionally, we can add parameters, like `{{$input}}`, to inject values in runtime.

We'll deep dive in Semantic Functions later and now we'll demonstrate a simplified way to create and run Semantic Function **inline**. With this method we can start structuring our application and transforming Prompts into functions. Please note that as we define the function we can also control the behavior of how we expect the kernel to run it, for example, controlling the max number of tokens in the response and the temperature. This we'll ensure that the function we'll run consistenly.

Therefore, in this step, we'll create an **inline semantic function** to classify a telco problem in different categories, like Mobile Internet, Fixed Internet, etc. This is based on the same example we run in Azure AI Studio and is one of the things we need to complete in our Call Center Analytics Solution.

In [9]:
# Set the prompt for categorizing based on the problem description
prompt = """
Problem Description: {{$input}}
Classify the problem description above as one of these categories: Fixed Internet, Mobile Internet, TV, Landline, Billing.
Only write the output category with no extra text.
Only write one category per problem description.
"""

# Create a semantic function for classifying the problem description
classify_call_function = kernel.create_semantic_function(prompt, 
                                                         max_tokens=10,
                                                         temperature=0.5)

Now that our Kernel has a new function to Categorize problems, we'll test it based on a problem description. We expected as a result one of the classifications previously defined in the prompt.

In [10]:
#Invoke the semantic function as a regular Python function
try:
    classification = classify_call_function(input)
    print(f"Classification: {classification}")

except Exception as e:
    print(f"Error: {e}")

Classification: 
Mobile Internet


Alternatively, we can also register the semantic function and already define important information to later reuse the function. The code below is an example and you can also use it to define semantic functions inline. This notation will be useful in the future as we create our AI Plugins, which we'll explore later.

In [11]:
from semantic_kernel import PromptTemplate, PromptTemplateConfig, SemanticFunctionConfig

#Define a configuration to control how the prompt will run in the model and define descriptions that can later be used by planners
prompt_config = PromptTemplateConfig(
    description="Classify a problem report in a call to the telco company based on provided categories.",
    type="completion",
    completion=PromptTemplateConfig.CompletionConfig(0.5, 0.0, 0.0, 0.0, 10),
    input=PromptTemplateConfig.InputConfig(
        parameters=[
            PromptTemplateConfig.InputParameter(
                name="input", description="The problem reported by the user that needs to be classified. Required parameter.", default_value=""
            )
        ]
    )
)

# Transform the Prompt into a PromptTemplate object
prompt_template = PromptTemplate(
    template=prompt,
    template_engine=kernel.prompt_template_engine,
    prompt_config=prompt_config
)

#Defines the semantic function configuration set
categorize_fn_config = SemanticFunctionConfig(prompt_config, prompt_template)

#Finally, register the new semantic function as part of an AI plugin. Later we'll explore this notation in more details
categorize_fn = kernel.register_semantic_function("CallCenterPlugin","Categorize", categorize_fn_config)


In [12]:
try:
    # Call the classify function with a problem description
    result = await kernel.run_async(categorize_fn, input_str=input)
    # Print the result
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {e}")

Result: 
Mobile Internet


#### Step 5 - Run Chat Completion prompts using Semantic Kernel

Models like **GPT-4** and **GPT-35-Turbo** were trained to accept inputs formatted as a **Conversation**. They use a different type of API called **Chat Completion**. Therefore, we need to make some adaptations to run prompts in these models as they require to use a different notation with messages considering three roles:
- **System:** used to to give instructions/context about to the model and how it should behave. Also known as System Message or Metaprompt.
- **User:** used to represent messages from the user. This is the messages we'll try to respond. Also refered as User Message.
- **Assistant:** used to represent messages/responses from the model. Also refered as Assistant Message.

Now, let's see how we can run Chat Completions in Semantic Kernel.

**Important:** As we'll now run **Chat Completion** operations, you need to review and run the cell below to recreate a `.env` file with a **Chat Completion** model. 

In [13]:
%%writefile .env
AZURE_OPENAI_DEPLOYMENT_NAME=""
AZURE_OPENAI_ENDPOINT=""
AZURE_OPENAI_API_KEY=""

Overwriting .env


To get started, we need to import the correct connector for AzureChatCompletion and add a Chat Completion model to the kernel. After that, it's important to correctly setup our prompt adding the different roles (System, Assistant, User). Eventually, you can use this notation to add more messages to act as few-shot examples to obtain better results from the model.

Please note that we're using as **System Message** the example we ran in Azure AI Studio for our Call Center Analytics solution to extract additional information from a conversation.

In [14]:
#Importing the Semantic Kernel SDK that will be used in this lab. 
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

#Instantiating the kernel
kernel = sk.Kernel()

# Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_chat_service("Chat Completion", 
                        AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key))

#Define a system message to model how the model should behave
system_message = """
You're an AI assistant that helps telco company to extract valuable information from their conversations by creating JSON files for each conversation transcription you receive. You always try to extract and format as a JSON:
1. Customer Name [name]
2. Customer Contact Phone [phone]
3. Main Topic of the Conversation [topic]
4. Customer Sentiment (Neutral, Positive, Negative)[sentiment]
5. How the Agent Handled the Conversation [agent_behavior]
6. What was the FINAL Outcome of the Conversation [outcome]
7. A really brief Summary of the Conversation [summary]

Only extract information that you're sure. If you're unsure, write "Unknown/Not Found" in the JSON file.
"""

#Defining the chat Completion parameters, like max_tokens, temperature, top_p, etc.
prompt_config = sk.PromptTemplateConfig.from_completion_parameters(
    max_tokens=500, temperature=0.7, top_p=0.5
)

#Defining a prompt template setup the Three Roles in a Chat Completion Operation. Here we're defining input for the User Message
prompt_template = sk.ChatPromptTemplate(
    "{{$input}}", kernel.prompt_template_engine, prompt_config
)

#Ensure our prompt template have at least a System Message
prompt_template.add_system_message(system_message)
#Optionally, we can load user and assistant messages to use as Few Shot examples
prompt_template.add_user_message("Hello, how are you?")
prompt_template.add_assistant_message("Hello! I can help you to extract valuable information from Call Center conversations.")

Now that we have our prompt ready and configured, we'll register it as a new inline Semantic Function and invoke it as a function. We should see important information extracted from our conversation in JSON format.

In [15]:
#Registering the Chat Completion Semantic Function
extract_fn_config = sk.SemanticFunctionConfig(prompt_config, prompt_template)
extract_fn = kernel.register_semantic_function("CallCenterPlugin", "ChatExtract", extract_fn_config)

try:
    # Create a context variable to hold the input and collecting the input from user
    context_vars = sk.ContextVariables()
    context_vars["input"] = input

    # Call the extract function with user input
    extraction = await kernel.run_async(extract_fn, input_vars=context_vars)
    print(f"Extraction: {extraction}")
    
except Exception as e:
    print(f"Error: {e}")   
    exit()

Extraction: {
  "name": "Martín Pérez",
  "phone": "011-4567-8910",
  "topic": "Problemas con el plan de datos móviles",
  "sentiment": "Negative",
  "agent_behavior": "El agente fue paciente, explicativo y ofreció una solución al problema del cliente",
  "outcome": "El cliente considerará la opción de cambiar su plan y llamará más tarde para confirmar",
  "summary": "El cliente, Martín Pérez, llamó porque tenía problemas con su plan de datos móviles. El agente, Juan, explicó que el cliente había consumido casi todo su límite de datos, lo que había reducido su velocidad de navegación. El cliente se sintió engañado porque no sabía que algunas aplicaciones consumen datos en segundo plano. Juan ofreció cambiar el plan del cliente a uno superior, con más GB de datos y mayor velocidad, por un costo adicional. El cliente dijo que lo pensaría y llamaría más tarde para confirmar."
}
